In [1]:
#Libraries
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.mlab as mlab

import sys
import math
import numexpr
import re
from __future__ import division

from datetime import date
import warnings

%matplotlib inline

## Step 1: Cleaning the original SCF data (city, state, zip3)

In [2]:
f = open("data/scf.csv")
clean_f = open('data/scf_clean.csv', 'w+')
remove_start = '.*\s{1}ST\s{1}|.*\s{1}DR\s{1}|.*\s{1}BLVD\s{1}|.*\s{1}WAY\s{1}|.*\s{1}HWY\s{1}|.*\s{1}PKWY\s{1}|.*\s{1}TRL\s{1}|.*\s{1}RD\s{1}|.*\s{1}LN\s{1}|.*\s{1}LOOP\s{1}|.*\s{1}PIKE\s{1}|.*\s{1}SQ\s{1}|.*\s{1}PL\s{1}|.*\s{1}AVE\s{1}'
remove_phone = '[0-9]{4}\s.*$'
final = '.*\s{1}[0-9]{3}'

for line in f:
    line = re.sub(remove_start,"",line)
    line = re.sub(remove_phone,"",line)
    line = re.findall(final,line)
    if len(line) == 0:
        clean_f.write('\n')
    else:
        clean_f.write("%s\n" % line[0])
f.close()
clean_f.close()
        

## Step 2: Format SCF data as CSV

In [3]:
f = open("data/scf_clean.txt")
clean_f = open("data/scf_final.csv", "w+")
zip3_regex = '\s{1}[0-9]{3}$'
state_regex = '\s{1}[A-Z]{2}$' 

for line in f:
    zip3 = re.findall(zip3_regex, line)
    line = re.sub(zip3_regex, "", line)
    state = re.findall(state_regex, line)
    line = re.sub(state_regex, "", line)
    if zip3 and state:
        clean_f.write(line.strip() +","+ state[0].strip() +","+ zip3[0].strip()+"\n")
        
f.close()
clean_f.close()
                    

## Step 3: Create full list of zipcode mappings to SCFs

I could not find a dataset mapping scf to counties and so manually added the "counties" column to the SCF dataset. However, SCFs service multiple zipcodes and so I will be "filling in the blank" zipcodes semi-manually (le sigh... data munging).

In [4]:
full_scf_df = pd.read_csv("data/full_scf_mappings.csv", usecols=["Zip3"])
full_mappings = []

# remove commas and list out "-" ranges so that we can split with a " " only
full_scf_mappings = [row.split(", ") for row in full_scf_df["Zip3"]]
for i in range(len(full_scf_mappings)):
    clean_zip_list = []
    for zip3 in full_scf_mappings[i]:
        if len(zip3) is 3:
            clean_zip_list.append(zip3)
        elif len(zip3) is 2:
            clean_zip_list.append("0"+zip3)
        else:
            first_zip = int(re.findall("^[0-9]{3}",zip3)[0])
            last_zip = int(re.findall("[0-9]{3}$",zip3)[0])
            for z in range(first_zip,last_zip+1):
                z = str(z)
                if len(z) is 3:
                    clean_zip_list.append(z)
                else:
                    prepend_zeros = 3-len(z)
                    for i in range(prepend_zeros):
                        z = "0" + z
                    clean_zip_list.append(z)
    full_mappings.append(clean_zip_list)   

## Step 4: Create missing zipcode mappings to SCFs in scf_counties.csv

In [5]:
scf_counties_df = pd.read_csv("data/scf_counties.csv",dtype={"Zip3":object})
scf_counties_df = scf_counties_df.sort(["Zip3"])
print scf_counties_df.head()    

all_scf_counties_df = pd.read_csv("data/all_scf_counties.csv")
print all_scf_counties_df.head()

            City State Zip3              County
0       MELVILLE    NY  005       Nassau County
1       SAN JUAN    PR  009  San Juan Municipio
2    SPRINGFIELD    MA  011      Hampden County
3     SHREWSBURY    MA  015    Worcester County
4  NORTH READING    MA  018    Middlesex County
   Unnamed: 0      City State  Zip3              County
0           0  MELVILLE    NY     5       Nassau County
1           4  SAN JUAN    PR     6  San Juan Municipio
2           5  SAN JUAN    PR     7  San Juan Municipio
3           6  SAN JUAN    PR     8  San Juan Municipio
4           7  SAN JUAN    PR     9  San Juan Municipio


In [6]:
rows_to_add = []
for scf_group in full_mappings:
    scf_center = scf_counties_df.loc[scf_counties_df['Zip3'].isin(scf_group)]
    city = scf_center.values[0][0]
    state = scf_center.values[0][1]
    county = scf_center.values[0][3]
    for zip3 in scf_group:
        rows_to_add.append({'City':city, "State":state, "Zip3":zip3, "County":county})

all_scf_counties_df = all_scf_counties_df.append(rows_to_add, ignore_index=True)
all_scf_counties_df = all_scf_counties_df.sort(["Zip3"])
print all_scf_counties_df.head()
all_scf_counties_df.to_csv("data/all_scf_counties.csv")

   Unnamed: 0      City State Zip3              County
0           0  MELVILLE    NY    5       Nassau County
1           4  SAN JUAN    PR    6  San Juan Municipio
2           5  SAN JUAN    PR    7  San Juan Municipio
3           6  SAN JUAN    PR    8  San Juan Municipio
4           7  SAN JUAN    PR    9  San Juan Municipio


## Step 5: Merge scp/county table with unemployment data

In [7]:
u05 = pd.read_csv("data/unemployment_data/u05.csv")
u06 = pd.read_csv("data/unemployment_data/u06.csv")
u07 = pd.read_csv("data/unemployment_data/u07.csv")
u08 = pd.read_csv("data/unemployment_data/u08.csv")
u09 = pd.read_csv("data/unemployment_data/u09.csv")
u10 = pd.read_csv("data/unemployment_data/u10.csv")
u11 = pd.read_csv("data/unemployment_data/u11.csv")
u12 = pd.read_csv("data/unemployment_data/u12.csv")
u13 = pd.read_csv("data/unemployment_data/u13.csv")
u14 = pd.read_csv("data/unemployment_data/u14.csv")

u_frames = [u06,u07,u08,u09,u10,u11,u12,u13,u14]
merged = u05
for frame in u_frames:
    merged = pd.merge(merged, frame, on='County', how='outer')
    
merged["State"] = merged["County"]

for i in range(len(merged)):
    if "District of Columbia" not in merged.loc[i, "County"]:
        merged.loc[i, "County"] = re.findall('^.*,',merged.loc[i, "County"])[0][:-1]
        merged.loc[i, "State"] = re.findall('[A-Z]{2}$',merged.loc[i, "State"])[0]

print merged.head()
merged.to_csv("data/unemployment_data/full_u.csv")

           County      2005      2006  2007  2008      2009  2010  2011  2012  \
0  Autauga County  3.8       3.3        3.3       5.1   9.7   8.9   8.3   7.0   
1  Baldwin County  4.0       3.2        3.1       4.6   9.8  10.0   9.0   7.5   
2  Barbour County  5.8       5.7        6.3       8.8  14.3  12.3  11.6  11.6   
3     Bibb County  4.5       4.2        4.1       5.8  13.3  11.4  10.5   8.5   
4   Blount County  3.6       3.2        3.2       4.7  10.0   9.8   8.7   6.9   

   2013  2014 State  
0   6.3   5.9    AL  
1   6.7   6.1    AL  
2  10.4  10.8    AL  
3   7.8   7.1    AL  
4   6.3   6.1    AL  


In [8]:
print merged.dtypes

County       object
2005         object
2006         object
2007        float64
2008        float64
2009        float64
2010        float64
2011        float64
2012        float64
2013        float64
2014        float64
State        object
dtype: object


## Step 6: Merge the unemployment data with the SCF data

In [9]:
unemployment_data = pd.read_csv("data/unemployment_data/full_u.csv")
unemployment_data = unemployment_data.drop("Unnamed: 0", axis=1)

scf_counties_data = pd.read_csv("data/all_scf_counties.csv", dtype={"Zip3":object})
scf_counties_data = scf_counties_data.drop("Unnamed: 0", axis=1)

print unemployment_data.head()
print scf_counties_data.head()

           County      2005      2006  2007  2008      2009  2010  2011  2012  \
0  Autauga County  3.8       3.3        3.3       5.1   9.7   8.9   8.3   7.0   
1  Baldwin County  4.0       3.2        3.1       4.6   9.8  10.0   9.0   7.5   
2  Barbour County  5.8       5.7        6.3       8.8  14.3  12.3  11.6  11.6   
3     Bibb County  4.5       4.2        4.1       5.8  13.3  11.4  10.5   8.5   
4   Blount County  3.6       3.2        3.2       4.7  10.0   9.8   8.7   6.9   

   2013  2014 State  
0   6.3   5.9    AL  
1   6.7   6.1    AL  
2  10.4  10.8    AL  
3   7.8   7.1    AL  
4   6.3   6.1    AL  
   Unnamed: 0.1      City State Zip3              County
0             0  MELVILLE    NY    5       Nassau County
1             4  SAN JUAN    PR    6  San Juan Municipio
2             5  SAN JUAN    PR    7  San Juan Municipio
3             6  SAN JUAN    PR    8  San Juan Municipio
4             7  SAN JUAN    PR    9  San Juan Municipio


In [10]:
scf_unemployment_data = pd.merge(scf_counties_data, unemployment_data, on=['County','State'], how='left')
print scf_unemployment_data.head()

   Unnamed: 0.1      City State Zip3              County      2005      2006  \
0             0  MELVILLE    NY    5       Nassau County  4.0       3.7        
1             4  SAN JUAN    PR    6  San Juan Municipio  8.0       7.3        
2             5  SAN JUAN    PR    7  San Juan Municipio  8.0       7.3        
3             6  SAN JUAN    PR    8  San Juan Municipio  8.0       7.3        
4             7  SAN JUAN    PR    9  San Juan Municipio  8.0       7.3        

   2007  2008      2009  2010  2011  2012  2013  2014  
0   3.7       4.7   6.9   7.1   6.8     7   5.9   4.8  
1   8.1       8.7  11.1  10.3  10.0     9   8.3   8.3  
2   8.1       8.7  11.1  10.3  10.0     9   8.3   8.3  
3   8.1       8.7  11.1  10.3  10.0     9   8.3   8.3  
4   8.1       8.7  11.1  10.3  10.0     9   8.3   8.3  


In [11]:
scf_unemployment_data.to_csv("data/scf_unemployment_data.csv")

## Step 7: Finally let's combine things!

In [16]:
# unemployment data
unemployment_data = pd.read_csv("data/scf_unemployment_data.csv", dtype={'Zip3':object})
unemployment_data = unemployment_data.drop(["Unnamed: 0", "Unnamed: 0.1", "City"], axis=1)
unemployment_data.columns = ["state","zip_code","county","u2005","u2006","u2007","u2008","u2009","u2010","u2011","u2012","u2013","u2014"]

# lending club data
lc_df = pd.read_csv("joined.csv", parse_dates=["issue_d"])
lc_df = lc_df.drop("Unnamed: 0", axis=1)
print lc_df.head()

  state zip_code              county     u2005     u2006  u2007  u2008  u2009  \
0    NY        5       Nassau County  4.0       3.7         3.7    4.7    6.9   
1    PR        6  San Juan Municipio  8.0       7.3         8.1    8.7   11.1   
2    PR        7  San Juan Municipio  8.0       7.3         8.1    8.7   11.1   
3    PR        8  San Juan Municipio  8.0       7.3         8.1    8.7   11.1   
4    PR        9  San Juan Municipio  8.0       7.3         8.1    8.7   11.1   

   u2010  u2011  u2012  u2013  u2014  
0    7.1    6.8      7    5.9    4.8  
1   10.3   10.0      9    8.3    8.3  
2   10.3   10.0      9    8.3    8.3  
3   10.3   10.0      9    8.3    8.3  
4   10.3   10.0      9    8.3    8.3  
      id  firstMissed censored   n  receivedAfterMissed  dueWhenFirstMissed  \
0  54734           -1     True  26                    0            0.000000   
1  55742           -1     True  36                    0            0.000000   
2  56413           -1     True  14        

In [17]:
# remove last 2 x's from zip_code column in lending club data
lc_df.zip_code = lc_df.zip_code.apply(lambda x: x[:-2])
print lc_df.zip_code.head()

0    941
1    112
2    956
3    100
4    777
Name: zip_code, dtype: object


In [18]:
# merge the economic and lending club data
economic_lc_data = pd.merge(lc_df, unemployment_data, on=['zip_code'], how='left')

In [6]:
# 814 problem rows that are null (this is due to actual issues with the zipcodes)
print np.unique(economic_lc_data[economic_lc_data["u2005"].isnull()].zip_code)
# '090' '091' '092' '093' '094' '096' '098' '340' '962' '963' '965' '969' all represent intl armed forces

['090' '091' '092' '093' '094' '096' '098' '340' '353' '429' '569' '621'
 '643' '663' '682' '851' '854' '861' '888' '909' '929' '962' '963' '965'
 '969']


In [10]:
# create smaller dataset to get the job done
economic_lc_tractable = economic_lc_data[["id","issue_d","u2005","u2006","u2007","u2008","u2009","u2010","u2011","u2012","u2013","u2014"]]

In [11]:
# create the column for unemployment rates for the county in the preceding year
economic_lc_tractable['county_unemployment_-1'] = [row["u" + str(row.issue_d.year-1)] for (i, row) in economic_lc_tractable.iterrows()]

In [13]:
# create the column for unemployment rates for the county 2 years ago
economic_lc_tractable['county_unemployment_-2'] = [row["u" + str(row.issue_d.year-2)] for (i, row) in economic_lc_tractable.iterrows()]

In [15]:
print economic_lc_tractable[["id","issue_d","county_unemployment_-1","county_unemployment_-2"]].dtypes

id                                 int64
issue_d                   datetime64[ns]
county_unemployment_-1            object
county_unemployment_-2            object
dtype: object


In [16]:
# merge the relevant unemployment rate data back into the dataset and cleanup
economic_lc_tractable = economic_lc_tractable.drop(["issue_d","u2005","u2006","u2007","u2008","u2009","u2010","u2011","u2012","u2013","u2014"], axis = 1)
economic_lc_data = pd.merge(economic_lc_data, economic_lc_tractable, on=['id'], how='left')
economic_lc_data = economic_lc_data.drop(["u2005","u2006","u2007","u2008","u2009","u2010","u2011","u2012","u2013","u2014"], axis = 1)
economic_lc_data.head()

,id,firstMissed,censored,n,receivedAfterMissed,dueWhenFirstMissed,member_id,loan_amnt,funded_amnt,funded_amnt_inv,...,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,credit_policy,state,county,county_unemployment_-1,county_unemployment_-2
0,54734,-1,True,26,0,0.000000,80364,25000,25000,19080.057198,...,789,785,0,NaN,1,1,CA,San Francisco County,5.3,4.2
1,55742,-1,True,36,0,0.000000,114426,7000,7000,672.803839,...,679,675,0,NaN,1,1,NY,Kings County,5.4,5.4
2,56413,-1,True,14,0,0.000000,129814,7000,7000,0.007494,...,499,0,0,NaN,1,0,CA,Yolo County,5.7,5.2
3,57167,4,False,7,0,23108.329138,137225,25000,25000,24725.000000,...,539,535,0,NaN,1,1,NY,New York County,7.4,8
4,57245,-1,True,36,0,0.000000,138150,1200,1200,1200.000000,...,709,705,0,NaN,1,1,TX,Jefferson County,9.5,6.6


In [19]:
# convert unemployment rates to float (rather than obj)
economic_lc_data[["county_unemployment_-1","county_unemployment_-2"]] = economic_lc_data[["county_unemployment_-1","county_unemployment_-2"]].convert_objects(convert_numeric=True)

# create column for if a borrow is an internationally stationed military person
intl_mil_zips = ['090','091','092','093','094','096','098','340','962','963','965','969']
economic_lc_data["intl_military"] = [ zipcode in intl_mil_zips for zipcode in economic_lc_data.zip_code ]

id                                      int64
firstMissed                             int64
censored                                 bool
n                                       int64
receivedAfterMissed                   float64
dueWhenFirstMissed                    float64
member_id                               int64
loan_amnt                               int64
funded_amnt                             int64
funded_amnt_inv                       float64
term                                   object
int_rate                               object
installment                           float64
grade                                  object
sub_grade                              object
emp_title                              object
emp_length                             object
home_ownership                         object
annual_inc                            float64
verification_status                    object
issue_d                        datetime64[ns]
loan_status                       

#### Write out data, with preceding 2 years of county level unemployment and intl military columns

In [24]:
economic_lc_data.to_csv("data/joined_economic_na.csv")

##Step 8: Fill in NA for Unemployment Data

In [126]:
unemployment_data = pd.read_csv("data/joined_economic_na.csv", dtype={"zip_code":str})
unemployment_data = unemployment_data.drop("Unnamed: 0", axis=1)
print unemployment_data.head()

      id  firstMissed censored   n  receivedAfterMissed  dueWhenFirstMissed  \
0  54734           -1     True  26                    0            0.000000   
1  55742           -1     True  36                    0            0.000000   
2  56413           -1     True  14                    0            0.000000   
3  57167            4    False   7                    0        23108.329138   
4  57245           -1     True  36                    0            0.000000   

   member_id  loan_amnt  funded_amnt  funded_amnt_inv      ...       \
0      80364      25000        25000     19080.057198      ...        
1     114426       7000         7000       672.803839      ...        
2     129814       7000         7000         0.007494      ...        
3     137225      25000        25000     24725.000000      ...        
4     138150       1200         1200      1200.000000      ...        

  last_fico_range_low collections_12_mths_ex_med  mths_since_last_major_derog  \
0                

In [127]:
null_1 = np.unique(unemployment_data[unemployment_data["county_unemployment_-1"].isnull()].zip_code)
null_2 = np.unique(unemployment_data[unemployment_data["county_unemployment_-2"].isnull()].zip_code)
null_zips = np.unique(np.concatenate((null_1, null_2), axis=0))
# '090' '091' '092' '093' '094' '096' '098' '340' '962' '963' '965' '969' all represent intl armed forces
print null_zips

['090' '091' '092' '093' '094' '096' '098' '340' '353' '429' '569' '621'
 '643' '663' '682' '700' '701' '704' '851' '854' '861' '888' '909' '929'
 '962' '963' '965' '969']


Set all unemployment data for military to 0 since military people are employed (assumed)

In [128]:
military_zips = ['090','091','092','093','094','096','098','340','962','963','965','969']
unemployment_data.loc[unemployment_data.zip_code.isin(military_zips), ["county_unemployment_-1","county_unemployment_-2"]] = 0

In [129]:
null_1 = np.unique(unemployment_data[unemployment_data["county_unemployment_-1"].isnull()].zip_code)
null_2 = np.unique(unemployment_data[unemployment_data["county_unemployment_-2"].isnull()].zip_code)
null_zips = np.unique(np.concatenate((null_1, null_2), axis=0))
# '090' '091' '092' '093' '094' '096' '098' '340' '962' '963' '965' '969' all represent intl armed forces
print null_zips

['353' '429' '569' '621' '643' '663' '682' '700' '701' '704' '851' '854'
 '861' '888' '909' '929']


This next part is SUPER manual

In [130]:
# 353 is Birmingham
#print unemployment_data[unemployment_data["zip_code"] == '353'] 
unemployment_data.loc[unemployment_data["zip_code"] == '353', 'state'] = "AL"
unemployment_data.loc[unemployment_data["zip_code"] == '353', "county"] = 'Jefferson County'

# issue_d was in 2010
unemployment_data.loc[unemployment_data.id == 605575, "county_unemployment_-1"] = 10.8 # 2009
unemployment_data.loc[unemployment_data.id == 605575, "county_unemployment_-2"] = 5.4 # 2008

# issue_d was in 2014
unemployment_data.loc[unemployment_data.id == 22273859, "county_unemployment_-1"] = 6.7 # 2013
unemployment_data.loc[unemployment_data.id == 22273859, "county_unemployment_-2"] = 7.4 # 2012

In [131]:
# 429 is unassigned in Kentucky
print unemployment_data[unemployment_data["zip_code"] == '429']
unemployment_data.loc[unemployment_data["zip_code"] == '429', 'state'] = "KY"

# not assigned, so replace with state averages for the years (issue_d = 2014)
unemployment_data.loc[unemployment_data.id == 35084398, "county_unemployment_-1"] = 8 # 2013
unemployment_data.loc[unemployment_data.id == 35084398, "county_unemployment_-2"] = 8.2 # 2012

              id  firstMissed censored  n  receivedAfterMissed  \
451461  35084398           -1     True  4                    0   

        dueWhenFirstMissed  member_id  loan_amnt  funded_amnt  \
451461                   0   37767742       2000         2000   

        funded_amnt_inv      ...      last_fico_range_low  \
451461             2000      ...                      735   

       collections_12_mths_ex_med  mths_since_last_major_derog policy_code  \
451461                          0                           56           1   

       credit_policy state county county_unemployment_-1  \
451461             1   NaN    NaN                    NaN   

        county_unemployment_-2 intl_military  
451461                     NaN         False  

[1 rows x 67 columns]


In [132]:
# 621 is unassigned in Illinois
print unemployment_data[unemployment_data["zip_code"] == '621']
unemployment_data.loc[unemployment_data["zip_code"] == '621', 'state'] = "IL"

# not assigned, so replace with state averages for the years (issue_d = 2013)
unemployment_data.loc[unemployment_data.id == 6157020, "county_unemployment_-1"] = 9 # 2012
unemployment_data.loc[unemployment_data.id == 6157020, "county_unemployment_-2"] = 9.7 # 2011

             id  firstMissed censored   n  receivedAfterMissed  \
157758  6157020           -1     True  18                    0   

        dueWhenFirstMissed  member_id  loan_amnt  funded_amnt  \
157758                   0    7639148      17700        17700   

        funded_amnt_inv      ...      last_fico_range_low  \
157758            17700      ...                      780   

       collections_12_mths_ex_med  mths_since_last_major_derog policy_code  \
157758                          0                          NaN           1   

       credit_policy state county county_unemployment_-1  \
157758             1   NaN    NaN                    NaN   

        county_unemployment_-2 intl_military  
157758                     NaN         False  

[1 rows x 67 columns]


In [133]:
# 643 is unassigned in Missouri
print unemployment_data[unemployment_data["zip_code"] == '643']
unemployment_data.loc[unemployment_data["zip_code"] == '643', 'state'] = "MO"

# not assigned, so replace with state averages for the years (issue_d = 2013)
unemployment_data.loc[unemployment_data.id == 3286699, "county_unemployment_-1"] = 7 # 2012
unemployment_data.loc[unemployment_data.id == 3286699, "county_unemployment_-2"] = 8.5 # 2011

             id  firstMissed censored   n  receivedAfterMissed  \
107230  3286699           12    False  20          2259.941077   

        dueWhenFirstMissed  member_id  loan_amnt  funded_amnt  \
107230         7185.563913    4059186       9750         9750   

        funded_amnt_inv      ...      last_fico_range_low  \
107230             9675      ...                      540   

       collections_12_mths_ex_med  mths_since_last_major_derog policy_code  \
107230                          0                           48           1   

       credit_policy state county county_unemployment_-1  \
107230             1   NaN    NaN                    NaN   

        county_unemployment_-2 intl_military  
107230                     NaN         False  

[1 rows x 67 columns]


In [134]:
# 663 is unassigned in Kansas
print unemployment_data[unemployment_data["zip_code"] == '663']
unemployment_data.loc[unemployment_data["zip_code"] == '663', 'state'] = "KS"

# not assigned, so replace with state averages for the years (issue_d = 2011)
unemployment_data.loc[unemployment_data.id == 1012890, "county_unemployment_-1"] = 7.1 # 2010
unemployment_data.loc[unemployment_data.id == 1012890, "county_unemployment_-2"] = 6.9 # 2009

# issue_d = 2014
unemployment_data.loc[unemployment_data.id == 37681100, "county_unemployment_-1"] = 5.3 # 2013
unemployment_data.loc[unemployment_data.id == 37681100, "county_unemployment_-2"] = 5.8 # 2012

              id  firstMissed censored   n  receivedAfterMissed  \
39133    1012890           21    False  24           1677.93504   
466969  37681100           -1     True   3              0.00000   

        dueWhenFirstMissed  member_id  loan_amnt  funded_amnt  \
39133         14176.602942    1240055      18550        18550   
466969            0.000000   40454021      33800        33800   

        funded_amnt_inv      ...      last_fico_range_low  \
39133             18500      ...                      535   
466969            33800      ...                      760   

       collections_12_mths_ex_med  mths_since_last_major_derog policy_code  \
39133                           0                          NaN           1   
466969                          0                          NaN           1   

       credit_policy state county county_unemployment_-1  \
39133              1   NaN    NaN                    NaN   
466969             1   NaN    NaN                    NaN   

  

In [135]:
# 682 is unassigned in Nebraska
print unemployment_data[unemployment_data["zip_code"] == '682'][["id","issue_d","state","county_unemployment_-1","county_unemployment_-2"]]
unemployment_data.loc[unemployment_data["zip_code"] == '682', 'state'] = "NE"

#not assigned, so replace with state averages for the years (issue_d = 2014)
unemployment_data.loc[unemployment_data.id == 32169439, "county_unemployment_-1"] = 3.8 # 2013
unemployment_data.loc[unemployment_data.id == 32169439, "county_unemployment_-2"] = 4.0 # 2012

              id     issue_d state  county_unemployment_-1  \
428837  32169439  2014-11-01   NaN                     NaN   

        county_unemployment_-2  
428837                     NaN  


In [136]:
# 854 is unassigned in AZ
print unemployment_data[unemployment_data["zip_code"] == '854'][["id","issue_d","state","county_unemployment_-1","county_unemployment_-2"]]
unemployment_data.loc[unemployment_data["zip_code"] == '854', 'state'] = "AZ"

#not assigned, so replace with state averages for the years (issue_d = 2014)
unemployment_data.loc[unemployment_data.id == 31377412, "county_unemployment_-1"] = 7.8 # 2013
unemployment_data.loc[unemployment_data.id == 31377412, "county_unemployment_-2"] = 8.4 # 2012

              id     issue_d state  county_unemployment_-1  \
422841  31377412  2014-10-01   NaN                     NaN   

        county_unemployment_-2  
422841                     NaN  


In [137]:
# 861 is unassigned in AZ
print unemployment_data[unemployment_data["zip_code"] == '861'][["id","issue_d","state","county_unemployment_-1","county_unemployment_-2"]]
unemployment_data.loc[unemployment_data["zip_code"] == '861', 'state'] = "AZ"

#not assigned, so replace with state averages for the years (issue_d = 2014)
unemployment_data.loc[unemployment_data.id == 23783637, "county_unemployment_-1"] = 7.8 # 2013
unemployment_data.loc[unemployment_data.id == 23783637, "county_unemployment_-2"] = 8.4 # 2012

              id     issue_d state  county_unemployment_-1  \
370443  23783637  2014-08-01   NaN                     NaN   

        county_unemployment_-2  
370443                     NaN  


In [138]:
# 909 is unassigned in CA
print unemployment_data[unemployment_data["zip_code"] == '909'][["id","issue_d","state","county_unemployment_-1","county_unemployment_-2"]]
unemployment_data.loc[unemployment_data["zip_code"] == '909', 'state'] = "CA"

#not assigned, so replace with state averages for the years (issue_d = 2014)
unemployment_data.loc[unemployment_data.id == 16982625, "county_unemployment_-1"] = 8.9 # 2013
unemployment_data.loc[unemployment_data.id == 16982625, "county_unemployment_-2"] = 10.4 # 2012

              id     issue_d state  county_unemployment_-1  \
318618  16982625  2014-06-01   NaN                     NaN   

        county_unemployment_-2  
318618                     NaN  


In [139]:
# 929 is unassigned in CA
print unemployment_data[unemployment_data["zip_code"] == '929'][["id","issue_d","state","county_unemployment_-1","county_unemployment_-2"]]
unemployment_data.loc[unemployment_data["zip_code"] == '929', 'state'] = "CA"

#not assigned, so replace with state averages for the years (issue_d = 2014)
unemployment_data.loc[unemployment_data.id == 29033233, "county_unemployment_-1"] = 8.9 # 2013
unemployment_data.loc[unemployment_data.id == 29033233, "county_unemployment_-2"] = 10.4 # 2012

              id     issue_d state  county_unemployment_-1  \
405962  29033233  2014-10-01   NaN                     NaN   

        county_unemployment_-2  
405962                     NaN  


In [152]:
# 700 is assigned in New Orleans, LA
print unemployment_data[unemployment_data["zip_code"] == '700'][["id","issue_d","state","county","county_unemployment_-1","county_unemployment_-2"]]
unemployment_data.loc[unemployment_data["zip_code"] == '700', 'state'] = "LA"
unemployment_data.loc[unemployment_data["zip_code"] == '700', 'county'] = "Orleans Parish"

# issue d = 2007
unemployment_data.loc[unemployment_data.id == 143102, "county_unemployment_-1"] = 4.6 # 2006
unemployment_data.loc[unemployment_data.id == 143102, "county_unemployment_-2"] = 7.0 # 2005

# issue d = 2007
unemployment_data.loc[unemployment_data.id == 143398, "county_unemployment_-1"] = 4.6 # 2006
unemployment_data.loc[unemployment_data.id == 143398, "county_unemployment_-2"] = 7.0 # 2005

# issue_d = 2008
unemployment_data.loc[unemployment_data.id == 286038, "county_unemployment_-2"] = 4.6 # 2005

Empty DataFrame
Columns: [id, issue_d, state, county, county_unemployment_-1, county_unemployment_-2]
Index: []


In [159]:
# 701 is assigned in New Orleans, LA
print unemployment_data[unemployment_data["zip_code"] == '701'][["id","issue_d","state","county","county_unemployment_-1","county_unemployment_-2"]]
unemployment_data.loc[unemployment_data["zip_code"] == '701', 'state'] = "LA"
unemployment_data.loc[unemployment_data["zip_code"] == '701', 'county'] = "Orleans Parish"

# issue d = 2007
unemployment_data.loc[unemployment_data.id == 84670, "county_unemployment_-1"] = 4.6 # 2006
unemployment_data.loc[unemployment_data.id == 84670, "county_unemployment_-2"] = 7.0 # 2005

# issue d = 2007
unemployment_data.loc[unemployment_data.id == 136316, "county_unemployment_-1"] = 4.6 # 2006
unemployment_data.loc[unemployment_data.id == 136316, "county_unemployment_-2"] = 7.0 # 2005

# issue d = 2008
unemployment_data.loc[unemployment_data.id == 69967, "county_unemployment_-2"] = 4.6 # 2006
# issue d = 2008
unemployment_data.loc[unemployment_data.id == 238989, "county_unemployment_-2"] = 4.6 # 2006
# issue d = 2008
unemployment_data.loc[unemployment_data.id == 304780, "county_unemployment_-2"] = 4.6 # 2006
# issue d = 2008
unemployment_data.loc[unemployment_data.id == 352080, "county_unemployment_-2"] = 4.6 # 2006
# issue d = 2008
unemployment_data.loc[unemployment_data.id == 365927, "county_unemployment_-2"] = 4.6 # 2006


              id     issue_d state          county  county_unemployment_-1  \
39         69967  2008-05-01    LA  Orleans Parish                     5.1   
62         84670  2007-06-01    LA  Orleans Parish                     4.6   
302       136316  2007-10-01    LA  Orleans Parish                     4.6   
961       238989  2008-02-01    LA  Orleans Parish                     5.1   
1634      304780  2008-08-01    LA  Orleans Parish                     5.1   
2102      352080  2008-07-01    LA  Orleans Parish                     5.1   
2662      365927  2008-12-01    LA  Orleans Parish                     5.1   
2989      372817  2009-06-01    LA  Orleans Parish                     6.0   
3545      383733  2009-03-01    LA  Orleans Parish                     6.0   
3572      384068  2009-03-01    LA  Orleans Parish                     6.0   
5231      427725  2009-07-01    LA  Orleans Parish                     6.0   
5318      429347  2009-07-01    LA  Orleans Parish              

In [168]:
# 704 is assigned in Hammond, LA
print unemployment_data[unemployment_data["zip_code"] == '704'][unemployment_data["county_unemployment_-2"].isnull()][["id","issue_d","state","county","county_unemployment_-1","county_unemployment_-2"]]

# issue d = 2008
unemployment_data.loc[unemployment_data.id == 244029, "county_unemployment_-2"] = 3.6 # 2006

Empty DataFrame
Columns: [id, issue_d, state, county, county_unemployment_-1, county_unemployment_-2]
Index: []


In [177]:
# 851 is assigned in Phoenix, Az
unemployment_data.loc[unemployment_data["zip_code"] == '851', 'state'] = "AZ"
unemployment_data.loc[unemployment_data["zip_code"] == '851', 'county'] = "Maricopa County"

unemployment_data["issue_d"] = pd.to_datetime(unemployment_data["issue_d"])
maricopa_unemp = {2005:4.0,2006:3.6,2007:3.3,2008:5.4,2009:9.1,2010:9.5,2011:8.6,2012:7.3,2013:6.6,2014:5.9}
unemployment_data.loc[unemployment_data["zip_code"] == '851', 'county_unemployment_-1'] = [maricopa_unemp[row.issue_d.year-1] for (i, row) in unemployment_data[unemployment_data["zip_code"] == '851'].iterrows()]
unemployment_data.loc[unemployment_data["zip_code"] == '851', 'county_unemployment_-2'] = [maricopa_unemp[row.issue_d.year-2] for (i, row) in unemployment_data[unemployment_data["zip_code"] == '851'].iterrows()]
print unemployment_data[unemployment_data["zip_code"] == '851'][["id","issue_d","state","county","county_unemployment_-1","county_unemployment_-2"]]

              id    issue_d state           county  county_unemployment_-1  \
5540      432918 2009-08-01    AZ  Maricopa County                     5.4   
7733      464609 2009-12-01    AZ  Maricopa County                     5.4   
12657     526511 2010-06-01    AZ  Maricopa County                     9.1   
12751     527996 2010-06-01    AZ  Maricopa County                     9.1   
16428     580864 2010-09-01    AZ  Maricopa County                     9.1   
16474     581383 2010-09-01    AZ  Maricopa County                     9.1   
16659     584180 2010-09-01    AZ  Maricopa County                     9.1   
17100     590278 2010-10-01    AZ  Maricopa County                     9.1   
19135     616722 2010-11-01    AZ  Maricopa County                     9.1   
20665     637978 2010-12-01    AZ  Maricopa County                     9.1   
23111     675763 2011-02-01    AZ  Maricopa County                     9.5   
25201     715703 2011-04-01    AZ  Maricopa County              

In [185]:
# 569 and 888 are not assigned anywhere, so we'll just assign them values
print unemployment_data[unemployment_data["zip_code"].isin(['569','888'])][["id","issue_d","state","county","county_unemployment_-1","county_unemployment_-2"]]

# issue d = 2013
unemployment_data.loc[unemployment_data.id == 9745413, "county_unemployment_-1"] = round(unemployment_data["county_unemployment_-1"].mean(),1) # 2012
unemployment_data.loc[unemployment_data.id == 9745413, "county_unemployment_-2"] = round(unemployment_data["county_unemployment_-2"].mean(),1) # 2011

# issue d = 2014
unemployment_data.loc[unemployment_data.id == 23853778, "county_unemployment_-1"] = round(unemployment_data["county_unemployment_-1"].mean(),1) # 2013
unemployment_data.loc[unemployment_data.id == 23853778, "county_unemployment_-2"] = round(unemployment_data["county_unemployment_-2"].mean(),1) # 2012

              id    issue_d state county  county_unemployment_-1  \
226143   9745413 2013-12-01   NaN    NaN                     7.7   
371315  23853778 2014-08-01   NaN    NaN                     7.7   

        county_unemployment_-2  
226143                     8.3  
371315                     8.3  


We have no more null unemployment data, so let's save to a csv again

In [191]:
unemployment_data.to_csv("joined_economic.csv")

IOError: [Errno 28] No space left on device

In [190]:
print len(unemployment_data)

525787


## Today's Goals

* Seasonality of features
* Research concordance index and survival model performance measures

## Get the Data

In [106]:
lc_df = pd.read_csv("joined.csv", parse_dates=["issue_d"])
lc_df = lc_df.drop("Unnamed: 0", axis=1)

/usr/local/lib/python2.7/dist-packages/pandas/io/parsers.py:1170: DtypeWarning: Columns (25) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)


In [109]:
#u_df = pd.read_csv("data/state_unemployment.csv")
#u_df = u_df.drop("Rank", axis=1)
print len(np.unique(lc_df.zip_code))
print lc_df.zip_code.min()
print lc_df.zip_code.max()

print len(np.unique(all_scf_counties_df.Zip3))
print all_scf_counties_df.Zip3.min()
print all_scf_counties_df.Zip3.max()


893
007xx
999xx
915
005
999


In [123]:
state_prefixes = {
        'AK': 'Alaska',
        'AL': 'Alabama',
        'AR': 'Arkansas',
        'AS': 'American Samoa',
        'AZ': 'Arizona',
        'CA': 'California',
        'CO': 'Colorado',
        'CT': 'Connecticut',
        'DC': 'District of Columbia',
        'DE': 'Delaware',
        'FL': 'Florida',
        'GA': 'Georgia',
        'GU': 'Guam',
        'HI': 'Hawaii',
        'IA': 'Iowa',
        'ID': 'Idaho',
        'IL': 'Illinois',
        'IN': 'Indiana',
        'KS': 'Kansas',
        'KY': 'Kentucky',
        'LA': 'Louisiana',
        'MA': 'Massachusetts',
        'MD': 'Maryland',
        'ME': 'Maine',
        'MI': 'Michigan',
        'MN': 'Minnesota',
        'MO': 'Missouri',
        'MP': 'Northern Mariana Islands',
        'MS': 'Mississippi',
        'MT': 'Montana',
        'NA': 'National',
        'NC': 'North Carolina',
        'ND': 'North Dakota',
        'NE': 'Nebraska',
        'NH': 'New Hampshire',
        'NJ': 'New Jersey',
        'NM': 'New Mexico',
        'NV': 'Nevada',
        'NY': 'New York',
        'OH': 'Ohio',
        'OK': 'Oklahoma',
        'OR': 'Oregon',
        'PA': 'Pennsylvania',
        'PR': 'Puerto Rico',
        'RI': 'Rhode Island',
        'SC': 'South Carolina',
        'SD': 'South Dakota',
        'TN': 'Tennessee',
        'TX': 'Texas',
        'UT': 'Utah',
        'VA': 'Virginia',
        'VI': 'Virgin Islands',
        'VT': 'Vermont',
        'WA': 'Washington',
        'WI': 'Wisconsin',
        'WV': 'West Virginia',
        'WY': 'Wyoming'
}
#state_prefixes = {v.upper(): k for k, v in state_prefixes.items()}
#u_df.State = u_df.State.apply(lambda state: state_prefixes[state])

In [125]:
#u_df.columns = ["addr_state","state_unemployment","state_unemployment_change"]
#u_df.head()

In [130]:
# full_df = pd.merge(lc_df, u_df, how="outer", on="addr_state")
print np.unique(lc_df.zip_code)

['007xx' '008xx' '010xx' '011xx' '012xx' '013xx' '014xx' '015xx' '016xx'
 '017xx' '018xx' '019xx' '020xx' '021xx' '022xx' '023xx' '024xx' '025xx'
 '026xx' '027xx' '028xx' '029xx' '030xx' '031xx' '032xx' '033xx' '034xx'
 '035xx' '036xx' '037xx' '038xx' '040xx' '041xx' '043xx' '044xx' '045xx'
 '049xx' '050xx' '051xx' '052xx' '053xx' '054xx' '056xx' '057xx' '058xx'
 '059xx' '060xx' '061xx' '062xx' '063xx' '064xx' '065xx' '066xx' '067xx'
 '068xx' '069xx' '070xx' '071xx' '072xx' '073xx' '074xx' '075xx' '076xx'
 '077xx' '078xx' '079xx' '080xx' '081xx' '082xx' '083xx' '084xx' '085xx'
 '086xx' '087xx' '088xx' '089xx' '090xx' '091xx' '092xx' '093xx' '094xx'
 '096xx' '098xx' '100xx' '101xx' '102xx' '103xx' '104xx' '105xx' '106xx'
 '107xx' '108xx' '109xx' '110xx' '111xx' '112xx' '113xx' '114xx' '115xx'
 '116xx' '117xx' '118xx' '119xx' '120xx' '121xx' '122xx' '123xx' '124xx'
 '125xx' '126xx' '127xx' '128xx' '129xx' '130xx' '131xx' '132xx' '133xx'
 '134xx' '135xx' '136xx' '137xx' '138xx' '139xx' '1

In [31]:
print lc_df.ix[0].values

[54734 -1 True 26 0.0 0.0 80364 25000 25000 19080.057198275401 ' 36 months'
 ' 11.89%' 829.10000000000002 'B' 'B4' nan '< 1 year' 'RENT' 85000.0
 'VERIFIED - income' Timestamp('2009-08-01 00:00:00') 'Fully Paid' 'n'
 'https://www.lendingclub.com/browse/loanDetail.action?loan_id=54734'
 'Due to a lack of personal finance education and exposure to poor financing skills growing up, I was easy prey for credit predators. I am devoted to becoming debt-free and can assure my lenders that I will pay on-time every time. I have never missed a payment during the last 16 years that I have had credit. '
 'debt_consolidation' 'Debt consolidation for on-time payer' '941xx' 'CA'
 19.48 0.0 'Feb-1994' 735 739 0.0 nan nan 10.0 0.0 28854 '52.1%' 42.0 'f'
 0.0 0.0 29324.32 21811.700000000001 25000.0 4324.3199999999997 0.0 0.0 0.0
 '2011-10-01' 7392.0799999999999 nan '2012-08-01' 789 785 0.0 nan 1 1]
